# Dev: Trial Not Started

**Goal**: Development code for using the n_previous_trial_not_started feature as a regressor.

Approach: add to base DMG for now as an argument 

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from multiglm.experiments.experiment_sigma_sweep import ExperimentSigmaSweep
from multiglm.features.design_matrix_generator import *
from multiglm.models.multiclass_logistic_regression import MultiClassLogisticRegression
from multiglm.data import ANIMAL_IDS

from multiglm.data.dataset_loader import DatasetLoader

sns.set_context("talk")
%load_ext autoreload
%autoreload 2

In [25]:
df_all = DatasetLoader(data_type="new_trained").load_data()

Loading data for animal ids:  ['W051', 'W060', 'W065', 'W066', 'W068', 'W072', 'W073', 'W074', 'W075', 'W078', 'W080', 'W081', 'W082', 'W083', 'W088', 'W089', 'W094']


In [29]:
df_all.groupby("animal_id").n_prev_trial_not_started.mean().mean()

0.034178431935407615

In [2]:
df = DatasetLoader(["W078"], data_type="new_trained").load_data()

Loading data for animal ids:  ['W078']


In [7]:
df.columns

Index(['animal_id', 'session_date', 'session_file_counter', 'rig_id',
       'training_stage', 's_a', 's_b', 'hit', 'violation', 'trial_not_started',
       's_a_sigma', 'rule', 'correct_side', 'violation_penalty_time',
       'error_penalty_time', 'delayed_reward_time', 'pre_stim_time',
       's_a_time', 'delay_time', 's_b_time', 'post_s_b_to_go_cue_time',
       'go_cue_time', 'fixation_time', 'trial_start_wait_time', 'l_water_vol',
       'r_water_vol', 'antibias_beta', 'antibias_right_prob',
       'using_psychometric_pairs', 'choice', 'session',
       'session_relative_to_old', 'n_prev_trial_not_started', 'trial'],
      dtype='object')

In [3]:
def calculate_prev_violation(group):
    session_boundaries_mask = group["session"].diff() == 0
    group["prev_violation"] = (
        group["violation"].shift() * session_boundaries_mask
    ).fillna(0)
    return group


df = df.groupby("animal_id").apply(calculate_prev_violation).reset_index(drop=True)

In [20]:
df["prev_trial_not_started"] = df.n_prev_trial_not_started != 0

In [16]:
df["prev_disengaged"] = (df.n_prev_trial_not_started != 0) | (
    df.prev_violation == 1
).astype(int)

In [19]:
X, y = DesignMatrixGeneratorFilteredHistory().generate_design_matrix(
    df, filter_params={"prev_violation": 0}, prev_disengaged=True
)

In [20]:
X.columns

Index(['bias', 'session', 's_a', 's_b', 'prev_sound_avg', 'prev_correct',
       'prev_choice', 'prev_disengaged'],
      dtype='object')

In [18]:
X.prev_trial_not_started.describe()

count    80582.000000
mean         0.000919
std          0.013260
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: prev_trial_not_started, dtype: float64